In [1]:
import sys
sys.path.append("../baseline_models")

In [ ]:
import numpy as np
import xarray as xr
import pandas as pd

import matplotlib.pyplot as plt
import glob


In [ ]:
from utils import data_path

# This doesn't work with the NorESM CMIP data for some reason
_Y = xr.open_dataset(data_path + 'outputs_ssp245.nc')

weights = np.cos(np.deg2rad(_Y['tas'].lat)).expand_dims(lon=144).assign_coords(lon=_Y.lon)

def global_mean(da):
    return da.weighted(weights).mean(['lat', 'lon'])

weights

In [ ]:
model_labels = [
 'CNRM-CM6-1',
 'ACCESS-CM2',
 'INM-CM5-0',
 'MIROC6',
 'HadGEM3-GC31-LL',
 'MRI-ESM2-0',
 'MCM-UA-1-0',
 'FGOALS-g3',
 'CMCC-CM2-SR5',
 'CanESM5',
 'EC-Earth3',
 'INM-CM4-8',
 'BCC-CSM2-MR',
 'CESM2',
 'MPI-ESM1-2-LR',
 'CESM2-WACCM',
 'NorESM2-MM',
 'CNRM-CM6-1-HR',
 'MIROC-ES2L',
 'EC-Earth3-Veg',
 'CNRM-ESM2-1'
]

In [ ]:
# Example NorESM grid
noresm = xr.open_mfdataset("../CMIP6/*_Ayr_NorESM2-LM_ssp245_r1i1p1f1_gn_2015-2100.nc").interp_like(weights)
# noresm = xr.open_mfdataset("/Users/watson-parris/Local data/ClimateBench/CMIP6/*_Ayr_NorESM2-LM_ssp245_r1i1p1f1_gn_2015-2100.nc").interp_like(weights)
noresm['diurnal_temperature_range'] = noresm['tasmax'] - noresm['tasmin']
# noresm = noresm.drop('time').assign_coords(year=noresm['time'].dt.year)
noresm.coords['time'] = noresm['time'].dt.year
noresm

In [ ]:
models = []
for model in model_labels:
    print(model)
    try:
        cmip_ds = xr.open_mfdataset(f"../CMIP6/*_Ayr_{model}_ssp245_*.nc", decode_times=True)
        piclim = xr.open_mfdataset(f"../CMIP6/*_Ayr_{model}_piControl_*.nc", decode_times=True).isel(time=slice(-85, None)).mean('time')
        # cmip_ds = xr.open_mfdataset(f"/Users/watson-parris/Local data/ClimateBench/CMIP6/*_Ayr_{model}_ssp245_*.nc", decode_times=True)
        # piclim = xr.open_mfdataset(f"/Users/watson-parris/Local data/ClimateBench/CMIP6/*_Ayr_{model}_piControl_*.nc", decode_times=True).isel(time=slice(-85, None)).mean('time')
        cmip_ds = cmip_ds - piclim
    #     print(cmip_ds['time'].dt.year)
        cmip_ds.coords['time'] = cmip_ds['time'].dt.year
#     print(cmip_ds['year'])
        if 'tasmin' in cmip_ds:
            cmip_ds['diurnal_temperature_range'] = cmip_ds['tasmax'] - cmip_ds['tasmin']
        else:
            cmip_ds['diurnal_temperature_range'] = xr.full_like(cmip_ds['tas'], np.NaN)
        models.append(cmip_ds.interp_like(weights))
    except Exception:
        print("Failed to read model")

In [ ]:
models[2]

In [ ]:
Y = xr.concat([model.assign_coords(model=label) for model, label in zip(models, model_labels)], 'model').compute()

In [ ]:
variables = ['tas', 'pr', 'diurnal_temperature_range']

R2E = pd.DataFrame({
    "Variability": {variable: global_mean(Y[variable].sel(time=slice(2080, None)).mean('time')).std().data / np.abs(global_mean(Y.mean('model')[variable].sel(time=slice(2080, None)).mean('time')).data) for variable in variables} 
})

NRMSE = pd.DataFrame({
    "Variability": {variable: global_mean(Y[variable].sel(time=slice(2080, None)).mean('time').std('model')).data/ np.abs(global_mean(Y.mean('model')[variable].sel(time=slice(2080, None)).mean('time')).data) for variable in variables}
})

variability_df = pd.concat([NRMSE, R2E, NRMSE+5*R2E], keys=['Spatial', 'Global', 'Total']).T.swaplevel(axis=1)[variables]



In [ ]:
variability_df